In [ ]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
import unicodedata
import re
import utils as utils
from calendar import monthrange

In [ ]:
source_db = "trusted"
source_tables = ["f_flujo_aereo","f_ocupacion_hotelera"]
db_name = "exploitation"
table_name = "f_ocupacion_ciudad"

In [ ]:
spark = utils.create_context()

In [ ]:
df_ocupacion_hotelera = utils.read_iceberg_table(spark,db_name,table_name)

In [ ]:
@udf(IntegerType())
def dias_mes(year,month):

    days = monthrange(year,month)[1]

    return days


In [ ]:
df_ocupacion_hotelera = (
    df_ocupacion_hotelera
    .filter(col("lugarResidencia") == "Total")
    .withColumn(
        "diasMes",
        dias_mes(col("año"),col("mes"))
    )
    .withColumn(
        "Tasaocupacion",
        when(
            (col("plazasEstimadas").isNotNull()) & (col("plazasEstimadas") > 0),
            (col("pernoctaciones") / (col("plazasEstimadas") * col("diasMes"))) * 100
        ).otherwise(None)
    )
)

